# Redis Script 

Followed [this site](https://realpython.com/python-redis/) to create this.

A simple walk through of using Redis with Python. I have a 9 node cluster so output and certain commands work differently than the above link

Before working on this Notebook, I installed Redis with ```brew install redis``` and ```pip install redis-py-cluster``` to get the redis-cli and redis tools for python

*Note:* I needed to update my zsh path to make the redis-cli command work. 

In [1]:
# first import redis
from rediscluster import RedisCluster
from redis import *

### Logging In

To log into the redis cluster, the cli command is ```redis-cli -c -h {ip of a node} -a {password}```

- -c means I am using as a client 
- -h is the hostname or ip address to connect to 
- -a is the password of the cluster
- -p is the port (defaults to 6379)

My login:
```redis-cli -c -h 10.61.141.31 -a SCSLzUMQm63Mu0B```
```
10.61.140.254:6379> PING
{return}> PONG
```

In [2]:
# let's login with python 

# my constants
PORT='6379'

PASSWORD=''
with open('pass.txt', 'r') as my_pass:
    PASSWORD=my_pass.readline().strip()

startup_nodes = []

# get hosts from file
with open('hosts.txt', 'r') as f:
    for host in f:
        startup_nodes.append({'host': host.strip(), 'port': PORT})

# Connect to Redis
rc = RedisCluster(startup_nodes=startup_nodes, decode_responses=True, 
                  password=PASSWORD, skip_full_coverage_check=True)




# verify I connected
rc.ping()

{'10.61.140.254:6379': True,
 '10.61.140.241:6379': True,
 '10.61.140.248:6379': True,
 '10.61.141.31:6379': True,
 '10.61.142.102:6379': True,
 '10.61.146.44:6379': True,
 '10.61.141.224:6379': True,
 '10.61.145.76:6379': True,
 '10.61.145.23:6379': True}

### Setting

To add something, use the ```SET``` command

```
10.61.141.31:6379> SET Bahamas Nassau
OK
```

In [3]:
# now we set!
rc.mset({'Norway':'Oslo'})

True

### Getting

To retrieve what you added, use the ```GET``` command 

```
10.61.141.31:6379> GET Norway
"Oslo"
```

In [4]:
rc.get('Norway')

'Oslo'

Redis needs keys that are *bytes, str, int, or float* so this should cause an error 

In [5]:
import datetime
today = datetime.date.today()
visitors = {'Saad', 'Isaac', 'Sam', 'Rachel'}
rc.sadd(today, *visitors)

DataError: Invalid input of type: 'date'. Convert to a byte, string or number first.

Convert the datetime to a string and it should work

In [6]:
str_today = today.isoformat()
rc.sadd(str_today, *visitors)

4

Get the visitors

In [7]:
rc.smembers(str_today)

{'Isaac', 'Rachel', 'Saad', 'Sam'}

## Applications with Redis

Let's make a little online store and use Redis to hold our data!

We will need to **first** flush the data

In [8]:
rc.flushall()

{'10.61.140.254:6379': True,
 '10.61.141.31:6379': True,
 '10.61.141.224:6379': True}

In [9]:
# verify the db is empty
rc.keys()

[]

In [10]:
import random

# set this so we can hard code some of our gets
random.seed(444)
hats = {"hat:{}".format(random.getrandbits(32)): value for value in (
    {
        "color": "black",
        "price": 49.99,
        "style": "fitted",
        "quantity": 1000,
        "npurchased": 0,
    },
    {
        "color": "maroon",
        "price": 59.99,
        "style": "hipster",
        "quantity": 500,
        "npurchased": 0,
    },
    {
        "color": "green",
        "price": 99.99,
        "style": "baseball",
        "quantity": 200,
        "npurchased": 0,
    })
}

for hat in hats:
    print(hat)

hat:1326692461
hat:1236154736
hat:56854717


In [11]:
# we will use a pipeline to do this 
with rc.pipeline() as pipe:
    for hat_id, hat in hats.items():
        pipe.hmset(hat_id, hat)
    pipe.execute()

# verify the db added our hats
rc.keys()

['hat:1236154736', 'hat:56854717', 'hat:1326692461']

In [12]:
# let's save 
rc.bgsave()

{'10.61.140.254:6379': True,
 '10.61.140.241:6379': True,
 '10.61.140.248:6379': True,
 '10.61.141.31:6379': True,
 '10.61.142.102:6379': True,
 '10.61.146.44:6379': True,
 '10.61.141.224:6379': True,
 '10.61.145.76:6379': True,
 '10.61.145.23:6379': True}

In [13]:
rc.hgetall('hat:56854717')

{'color': 'green',
 'price': '99.99',
 'style': 'baseball',
 'quantity': '200',
 'npurchased': '0'}

#### Simulate a purchase

To do this, we need to increment the ```npurchased``` by 1 and decrement the ```quantity``` by 1

In [14]:
rc.hincrby('hat:56854717', 'npurchased', 1)
rc.hincrby('hat:56854717', 'quantity', -1)
rc.hgetall('hat:56854717')

{'color': 'green',
 'price': '99.99',
 'style': 'baseball',
 'quantity': '199',
 'npurchased': '1'}

### Now to do business with our stock of hats 🧢

1. We have to check if the Hat is in stock
2. If it is in stock then we have to *sell* it
3. Watch for race conditions 

To watch for race conditions, we can use a transaction block to keep the action atomic 

```
 MULTI
 HINCRBY 56854717 quantity -1
 HINCRBY 56854717 npurchased 1
 EXEC
```

In [15]:
class Out_Of_Stock(Exception):
    '''Raised when out of stock'''

def buy_item(r: RedisCluster, item_id: int):
    amount_left = r.hget(item_id, 'quantity')
    # in stock
    if int(amount_left) > 0:
        r.hincrby(item_id, "quantity", -1)
        r.hincrby(item_id, "npurchased", 1)
    else:
        raise Out_Of_Stock('Sorry! {} is out of stock'.format(item_id))
            

**A NOTE** 

So in order to work with clusters, we run into race conditions. Imagine a user is looking at a special hat to buy and he calls ```buy_item({hat id})``` and there is a check to see if there are enough hats left for him to buy. What if there is only 1 hat left and 2 people call ```buy_item``` at the same time. Now who gets the hat. That is where the ```watch``` method comes into play. This method watches an item so that when someone calls ```buy_item``` on an item, that item will be tracked so if there is anyone else trying to also buy that item there will be an error. The problem we are having here is in order to work with clusters, one must use the ```RedisCluster``` Module and this module does not have this ```watch``` method implemented yet. The ```RedisCluster``` module can be modified to implement a ```watch``` method. As far as I know, the ```RedisCluster``` module is written atop the ```Redis``` module so we could use ```Redis.watch``` to implement ```RedisCluster.watch```. In order to do this, we would begin by finding which node the item is stored on and then watching that individually item. If the ```watch``` method was implemented then this code would work. 

```
def buy_item(r: RedisCluster, item_id: int) -> None:
    with r.pipeline() as pipe:
        error_count = 0
        while True:
            try:
                pipe.watch(item_id)
                amount_left = r.hget(item_id, 'quantity')
                
                # in stock
                if amount_left > 0:
                    pipe.multi()
                    pipe.hincrby(item_id, "quantity", -1)
                    pipe.hincrby(item_id, "npurchased", 1)
                    pipe.execute()
                    break
                else:
                    pipe.unwatch()
                    raise Out_Of_Stock('Sorry! {} is out of stock'.format(item_id))
            
            except redis.WatchError:
                error_count += 1
                logging.warning(
                    'WatchError #%d: %s; retrying',
                    error_count, itemid
                )
```
Till the method is written, we will just assume that one user can be in our online store at a time and there are no malicious actors.

Methods ```multi``` and ```execute``` are also not implemented so we will have a very stripped down version  

In [16]:
# let's try it
buy_item(rc, 'hat:56854717')
rc.hmget('hat:56854717', 'quantity', 'npurchased')

['198', '2']

In [17]:
buy_item(rc, 'hat:56854717')
rc.hmget('hat:56854717', 'quantity', 'npurchased')

['197', '3']